# Wheelchair Accessibility in Tashkent (OSM Data)

This analysis shows wheelchair accessibility based on OpenStreetMap tags.  
No data does not mean no accessibility. It often means the place is not yet mapped.

In [ ]:
# Import required libraries
import requests
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as ctx
from shapely.geometry import Point
import numpy as np
from matplotlib.patches import Rectangle
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully")

## 1. Fetch Wheelchair Accessibility Data from OpenStreetMap

In [ ]:
# Define central Tashkent bounding box (to avoid timeout)
# Focus on downtown area around Amir Timur Square
bbox = {
    'south': 41.28,
    'west': 69.20,
    'north': 41.35,
    'east': 69.32
}

print(f"Query area: Central Tashkent")
print(f"Bounds: {bbox['west']}-{bbox['east']}°E, {bbox['south']}-{bbox['north']}°N")

# Overpass API query for wheelchair accessibility tags
overpass_query = f"""
[out:json][timeout:30];
(
  node["wheelchair"~"yes|no|limited"]({bbox['south']},{bbox['west']},{bbox['north']},{bbox['east']});
);
out;
"""

# Fetch data from Overpass API
overpass_url = "https://overpass-api.de/api/interpreter"
print("\nFetching wheelchair accessibility data from OpenStreetMap...")

response = requests.post(overpass_url, data=overpass_query)
data = response.json()

print(f"Fetched {len(data['elements'])} features with wheelchair tags")

## 2. Process and Categorize Data

In [ ]:
# Convert OSM data to GeoDataFrame
features = []

for element in data['elements']:
    if element['type'] == 'node' and 'lat' in element and 'lon' in element:
        features.append({
            'name': element.get('tags', {}).get('name', 'Unnamed'),
            'amenity': element.get('tags', {}).get('amenity', 
                      element.get('tags', {}).get('building',
                      element.get('tags', {}).get('shop', 'Unknown'))),
            'wheelchair': element['tags']['wheelchair'],
            'geometry': Point(element['lon'], element['lat'])
        })

# Create GeoDataFrame
gdf = gpd.GeoDataFrame(features, crs='EPSG:4326')

# Categorize by accessibility level
accessible = gdf[gdf['wheelchair'] == 'yes']
limited = gdf[gdf['wheelchair'] == 'limited']
not_accessible = gdf[gdf['wheelchair'] == 'no']

print(f"\nData categorized:")
print(f"  Accessible (yes):        {len(accessible)}")
print(f"  Limited accessibility:   {len(limited)}")
print(f"  Not accessible (no):     {len(not_accessible)}")
print(f"  Total:                   {len(gdf)}")

## 3. Calculate Accessibility Metrics

In [ ]:
# Calculate percentages
total = len(gdf)

if total > 0:
    pct_accessible = (len(accessible) / total) * 100
    pct_limited = (len(limited) / total) * 100
    pct_not_accessible = (len(not_accessible) / total) * 100
    
    print("\n" + "="*50)
    print("ACCESSIBILITY METRICS - CENTRAL TASHKENT")
    print("="*50)
    print(f"\nTotal tagged locations:      {total}")
    print(f"\nAccessibility breakdown:")
    print(f"  Fully accessible:          {len(accessible):3d} ({pct_accessible:5.1f}%)")
    print(f"  Limited accessibility:     {len(limited):3d} ({pct_limited:5.1f}%)")
    print(f"  Not accessible:            {len(not_accessible):3d} ({pct_not_accessible:5.1f}%)")
    
    # Calculate amenity breakdown
    print(f"\nTop amenity types:")
    amenity_counts = gdf['amenity'].value_counts().head(5)
    for amenity, count in amenity_counts.items():
        print(f"  {amenity:20s}: {count}")
    
    # Calculate accessibility by amenity type
    print(f"\nAccessibility by top amenity types:")
    for amenity in amenity_counts.index[:3]:
        amenity_data = gdf[gdf['amenity'] == amenity]
        amenity_yes = len(amenity_data[amenity_data['wheelchair'] == 'yes'])
        amenity_total = len(amenity_data)
        if amenity_total > 0:
            amenity_pct = (amenity_yes / amenity_total) * 100
            print(f"  {amenity:20s}: {amenity_yes}/{amenity_total} ({amenity_pct:5.1f}% accessible)")
    
    print("\n" + "="*50)
else:
    print("No data found")

## 4. Spatial Distribution Analysis

In [ ]:
# Calculate spatial density (locations per square km)
if len(gdf) > 0:
    # Calculate area of bounding box in km²
    # 1 degree latitude ≈ 111 km, 1 degree longitude at 41°N ≈ 83 km
    lat_range = bbox['north'] - bbox['south']
    lon_range = bbox['east'] - bbox['west']
    area_km2 = (lat_range * 111) * (lon_range * 83)
    
    density = len(gdf) / area_km2
    density_accessible = len(accessible) / area_km2
    
    print(f"\nSpatial metrics:")
    print(f"  Study area:                {area_km2:.1f} km²")
    print(f"  Tagged locations density:  {density:.2f} per km²")
    print(f"  Accessible locations:      {density_accessible:.2f} per km²")
    
    # Calculate center of accessible locations
    if len(accessible) > 0:
        center_accessible = accessible.geometry.unary_union.centroid
        print(f"\n  Center of accessible locations: {center_accessible.y:.4f}°N, {center_accessible.x:.4f}°E")

## 5. Create Minimalist Map Visualization

In [ ]:
# Create figure with white background
fig, ax = plt.subplots(figsize=(12, 10), facecolor='white')
ax.set_facecolor('#f8f8f8')

# Plot data with simple symbols
if len(accessible) > 0:
    accessible.plot(ax=ax, color='#22c55e', markersize=60, 
                   marker='o', label='Accessible', zorder=3,
                   edgecolor='white', linewidth=0.5)

if len(limited) > 0:
    limited.plot(ax=ax, color='#eab308', markersize=60,
                marker='o', label='Limited', zorder=2,
                edgecolor='white', linewidth=0.5)

if len(not_accessible) > 0:
    not_accessible.plot(ax=ax, color='#ef4444', markersize=80,
                       marker='x', label='Not Accessible', zorder=4,
                       linewidth=2)

# Add basemap with low opacity (major roads visible)
try:
    ctx.add_basemap(ax, crs=gdf.crs.to_string(), 
                   source=ctx.providers.OpenStreetMap.Mapnik,
                   alpha=0.15, zorder=1)
except:
    print("Note: Basemap not added (requires internet connection)")

# Set map extent
ax.set_xlim(bbox['west'], bbox['east'])
ax.set_ylim(bbox['south'], bbox['north'])

# Remove axis ticks and labels for minimalist look
ax.set_xticks([])
ax.set_yticks([])
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)

# Add title
plt.title('Wheelchair Accessibility in Tashkent (OSM Data)', 
         fontsize=16, fontweight='600', pad=20, color='#222222')

# Add legend in bottom right
legend = ax.legend(loc='lower right', frameon=True, 
                  facecolor='white', edgecolor='#e0e0e0',
                  fontsize=11, markerscale=0.8)

# Add note about data gaps
note_text = "No data does not mean no accessibility.\nIt often means the place is not yet mapped."
ax.text(0.02, 0.02, note_text, transform=ax.transAxes,
       fontsize=9, style='italic', color='#666666',
       bbox=dict(boxstyle='round', facecolor='white', 
                edgecolor='#e0e0e0', alpha=0.95),
       verticalalignment='bottom')

# Add statistics box in top left
if total > 0:
    stats_text = f"Tagged locations: {total}\n"
    stats_text += f"Accessible: {len(accessible)} ({pct_accessible:.1f}%)\n"
    stats_text += f"Limited: {len(limited)} ({pct_limited:.1f}%)\n"
    stats_text += f"Not accessible: {len(not_accessible)} ({pct_not_accessible:.1f}%)"
    
    ax.text(0.02, 0.98, stats_text, transform=ax.transAxes,
           fontsize=10, color='#333333',
           bbox=dict(boxstyle='round', facecolor='white',
                    edgecolor='#e0e0e0', alpha=0.95),
           verticalalignment='top', family='monospace')

plt.tight_layout()
plt.savefig('accessibility_map.png', dpi=300, bbox_inches='tight', 
           facecolor='white', edgecolor='none')
plt.show()

print("\nMap saved as 'accessibility_map.png'")

## 6. Summary Statistics Chart

In [ ]:
# Create simple bar chart of accessibility breakdown
if total > 0:
    fig, ax = plt.subplots(figsize=(8, 5), facecolor='white')
    
    categories = ['Accessible', 'Limited', 'Not Accessible']
    counts = [len(accessible), len(limited), len(not_accessible)]
    colors = ['#22c55e', '#eab308', '#ef4444']
    
    bars = ax.bar(categories, counts, color=colors, edgecolor='white', linewidth=2)
    
    # Add count labels on bars
    for bar, count, pct in zip(bars, counts, [pct_accessible, pct_limited, pct_not_accessible]):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
               f'{count}\n({pct:.1f}%)',
               ha='center', va='bottom', fontsize=11, fontweight='600')
    
    ax.set_ylabel('Number of Locations', fontsize=12, color='#333333')
    ax.set_title('Wheelchair Accessibility Distribution - Central Tashkent', 
                fontsize=13, fontweight='600', pad=15, color='#222222')
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.set_facecolor('white')
    ax.grid(axis='y', alpha=0.3, linestyle='--')
    
    plt.tight_layout()
    plt.savefig('accessibility_stats.png', dpi=300, bbox_inches='tight',
               facecolor='white', edgecolor='none')
    plt.show()
    
    print("Chart saved as 'accessibility_stats.png'")

## 7. Export Data

In [ ]:
# Save processed data to CSV
if len(gdf) > 0:
    # Create export dataframe with coordinates
    export_df = gdf.copy()
    export_df['longitude'] = export_df.geometry.x
    export_df['latitude'] = export_df.geometry.y
    export_df = export_df.drop(columns=['geometry'])
    
    export_df.to_csv('accessibility_data.csv', index=False)
    print(f"Data exported to 'accessibility_data.csv' ({len(export_df)} records)")
    
    # Display sample
    print("\nSample data:")
    print(export_df.head(10))